In [14]:
import pandas as pd
import numpy as np
from collections import Counter
import itertools
import networkx as nx
import matplotlib.pyplot as plt
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import functions as fn
import warnings
warnings.filterwarnings('ignore')

AttributeError: module 'numpy._core._multiarray_umath' has no attribute '_blas_supports_fpe'

In [4]:
df_PLTCM = pd.read_csv('data/pltcm_manipulated_59604_rev1.csv', header=None)
pd.read_csv('data/pltcm_manipulated_59604_rev1.csv', names=fn.columns_PLTCM, header=None)

,EVENT_ID,PROGRAM_ID,1,2,3,4,5,6,INPUT_WIDTH,INPUT_THICKNESS,...,TARGET_WIDTH,THICKNESS_HSM,THICKNESS,TRIM_WIDTH,WIDTH,MATERIAL_ID,PIECE_ID,WEIGHT,LENGTH,INPUT_LENGTH
0,1,3609,0,0,0,0,0,0,1264.0446,2.55459,...,1222.6404,2.55459,0.477354,1224.67,1222.6381,318580,17167961-03000,13.57540,2998.57,543.95600
1,2,3609,0,0,0,0,0,0,1129.5043,2.73491,...,1089.0250,2.73491,0.564271,1091.06,1089.0281,323346,17172941-01000,21.04121,4387.14,875.23380
2,3,3609,0,0,0,0,0,0,1096.1252,2.71956,...,1057.3627,2.71956,0.614289,1059.39,1057.3580,323357,17172961-04000,15.52353,3061.01,672.14825
3,4,3609,0,0,0,0,0,0,1129.7084,2.73506,...,1089.0250,2.73506,0.564197,1091.06,1089.0281,323344,17172921-05000,21.02293,4386.41,875.66516
4,5,3609,0,0,0,0,0,0,1569.3829,2.63399,...,1528.7625,2.63399,0.569115,1529.56,1528.7650,318740,17168341-04000,17.28409,2558.96,536.53850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59599,59600,22800,0,0,0,0,0,0,1261.3135,5.00986,...,1222.3738,5.00986,1.431855,1222.90,1222.3717,1117991,20165941-06000,17.57078,1271.65,371.10983
59600,59601,22800,0,0,0,0,0,0,1261.1356,4.65018,...,1222.3738,4.65018,1.146508,1222.90,1222.3717,1117999,20165961-07000,17.78187,1612.51,404.14432
59601,59602,22800,0,0,0,0,0,0,1263.5488,2.73054,...,1212.2137,2.73054,0.512399,1212.74,1212.2117,1118016,20166001-07000,17.53634,3656.06,685.52203
59602,59603,22800,0,0,0,0,0,0,1280.8510,2.52041,...,1242.1870,2.52041,0.470304,1245.74,1242.1840,1117987,20165941-02000,20.33075,4472.55,840.73800


In [5]:
df = df_PLTCM
feature = 8 # input width
df_binned = pd.DataFrame()
df_binned['sequence_id'] = df[1]
df_binned[feature] = df[feature]
df_binned['bins_fxd_step'] = fn.binning_fxd_step(df[feature], 11)
df_binned['bins_fxd_bucket'] = fn.binning_fxd_count(df[feature], 200)
df_binned

,sequence_id,8,bins_fxd_step,bins_fxd_bucket
0,3609,1264.0446,"(1258, 1269)","(1263, 1265)"
1,3609,1129.5043,"(1126, 1137)","(1127, 1131)"
2,3609,1096.1252,"(1093, 1104)","(1094, 1098)"
3,3609,1129.7084,"(1126, 1137)","(1127, 1131)"
4,3609,1569.3829,"(1566, 1577)","(1569, 1570)"
...,...,...,...,...
59599,22800,1261.3135,"(1258, 1269)","(1260, 1262)"
59600,22800,1261.1356,"(1258, 1269)","(1260, 1262)"
59601,22800,1263.5488,"(1258, 1269)","(1263, 1264)"
59602,22800,1280.8510,"(1280, 1291)","(1280, 1282)"


In [13]:
frequent_itemsets = apriori(df['sequence_id'], min_support=0.6, use_colnames=False)

AttributeError: 'numpy.dtypes.Int64DType' object has no attribute 'apply'

In [6]:
df = df_binned
aimbaskets = df.groupby('sequence_id')['bins_fxd_step'].apply(list).reset_index()
aimbaskets = aimbaskets['bins_fxd_step'].to_list()
binningmembers = list(set(df['bins_fxd_step'].to_list()))
binningmembers.sort()

singlesupportvalues = []
for j in range(len(aimbaskets)):
    a = []
    res = Counter(aimbaskets[j])
    for i in binningmembers:
        a.append(res[i]/len(aimbaskets[j]))
    singlesupportvalues.append(a)

print(len(aimbaskets))
print(len(singlesupportvalues))
print(len(binningmembers))
print(len(singlesupportvalues[1]))

524
524
85
85


In [7]:
pairs = list(itertools.product(binningmembers, repeat=2))
pairs

[((949, 961), (949, 961)),
 ((949, 961), (961, 972)),
 ((949, 961), (972, 983)),
 ((949, 961), (983, 994)),
 ((949, 961), (994, 1005)),
 ((949, 961), (1005, 1016)),
 ((949, 961), (1016, 1027)),
 ((949, 961), (1027, 1038)),
 ((949, 961), (1038, 1049)),
 ((949, 961), (1049, 1060)),
 ((949, 961), (1060, 1071)),
 ((949, 961), (1071, 1082)),
 ((949, 961), (1082, 1093)),
 ((949, 961), (1093, 1104)),
 ((949, 961), (1104, 1115)),
 ((949, 961), (1115, 1126)),
 ((949, 961), (1126, 1137)),
 ((949, 961), (1137, 1148)),
 ((949, 961), (1148, 1159)),
 ((949, 961), (1159, 1170)),
 ((949, 961), (1170, 1181)),
 ((949, 961), (1181, 1192)),
 ((949, 961), (1192, 1203)),
 ((949, 961), (1203, 1214)),
 ((949, 961), (1214, 1225)),
 ((949, 961), (1225, 1236)),
 ((949, 961), (1236, 1247)),
 ((949, 961), (1247, 1258)),
 ((949, 961), (1258, 1269)),
 ((949, 961), (1269, 1280)),
 ((949, 961), (1280, 1291)),
 ((949, 961), (1291, 1302)),
 ((949, 961), (1302, 1313)),
 ((949, 961), (1313, 1324)),
 ((949, 961), (1324, 13

In [8]:
#df_PLTCM.iloc[[0,1,2,3,4,67,68,69,70,71,72,73,74,75],[1,8]]

def fun(df, col_key, col_value):
         keys, values = df.sort_values(col_key).values.T
         ukeys, index = np.unique(keys, True)
         arrays = np.split(values, index[1:])
         df2 = pd.DataFrame({col_key:ukeys, col_value:[list(a) for a in arrays]}, )
         return df2

dfG = fun(df_PLTCM[[1,8]], 1, 8)
dfG

,1,8
0,3609.0,"[1264.0446, 1129.5043, 1096.1252, 1129.7084, 1..."
1,3657.0,"[1259.9236, 1287.8539, 1284.3087, 1288.9408, 1..."
2,3681.0,"[1560.9393, 1259.6716, 1258.5481, 1217.0742, 1..."
3,3710.0,"[1569.8542, 1565.6652, 1571.4286, 1395.5007, 1..."
4,3721.0,"[1275.4269, 1261.335, 1576.0254, 1272.8102, 12..."
...,...,...
519,22519.0,"[1351.1455, 1541.3822, 1507.0881, 1305.5166, 1..."
520,22552.0,"[1265.6625, 1665.4911, 1266.5665, 1260.4731, 1..."
521,22652.0,"[1572.6578, 1575.6455, 1574.8002, 1571.4368, 1..."
522,22757.0,"[1549.4031, 1258.1987, 1761.4673, 1390.3076, 1..."


In [9]:
#df_PLTCM.groupby(1)[8].apply(list)
#list(df_PLTCM.groupby(1)[8].apply(list).values)
#aimbaskets = df_PLTCM.groupby(1)[8].apply(list).reset_index(name=names_columns[8])
aimbaskets = df_PLTCM.groupby(1)[8].apply(list).reset_index()
aimbaskets = aimbaskets[8].to_list()
binningmembers = list(set(df_PLTCM[8].to_list()))
binningmembers.sort()

singlesupportvalues = []
for j in range(len(aimbaskets)):
    a = []
    res = Counter(aimbaskets[j])
    for i in binningmembers:
        a.append(res[i]/len(aimbaskets[j]))
    singlesupportvalues.append(a)

print(len(aimbaskets))
print(len(singlesupportvalues))
print(len(binningmembers))
print(len(singlesupportvalues[1]))

524
524
57840
57840


In [10]:
binningmembers

[950.6825,
 951.0327,
 951.46674,
 951.9694,
 952.6057,
 952.86,
 953.0912,
 954.31793,
 954.37683,
 954.5889,
 955.1309,
 955.19385,
 955.4493,
 955.4771,
 955.4941,
 955.67267,
 955.76605,
 955.7739,
 955.88116,
 955.92645,
 955.9931,
 955.9957,
 955.9966,
 956.03204,
 956.04944,
 956.069,
 956.08875,
 956.0909,
 956.11786,
 956.26666,
 956.26874,
 956.2891,
 956.3302,
 956.3525,
 956.49634,
 956.55835,
 956.5912,
 956.6224,
 956.7184,
 956.7593,
 956.77893,
 956.7818,
 956.8903,
 956.8963,
 956.93365,
 956.9939,
 956.997,
 957.02057,
 957.0451,
 957.1437,
 957.2848,
 957.7288,
 957.75366,
 957.8488,
 958.09924,
 958.3311,
 958.3414,
 958.3423,
 958.6832,
 958.78564,
 958.8145,
 958.8394,
 958.87225,
 958.8766,
 958.8773,
 958.91675,
 958.97797,
 958.99225,
 958.9938,
 958.99414,
 959.06805,
 959.096,
 959.11053,
 959.17645,
 959.2977,
 959.38226,
 959.38806,
 959.4819,
 959.4846,
 959.5045,
 959.5783,
 959.64624,
 959.7789,
 959.79065,
 959.82764,
 959.8305,
 959.9052,
 959.9779,
 9

In [11]:
test = np.array(singlesupportvalues)
test2=test.transpose().tolist()
len(test2[1])

524

In [15]:
#list(itertools.product(range(1,3), repeat=2))
#pairs = list(itertools.product(binningmembers, repeat=2))
#pairs

In [ ]:
frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)